# Deep Learning Fundamentals<br />
<h2><p style="color:darkred">4 - Image classification with custom estimators</p></h2>

## Introduction

Consider the application of supervised learning techniques to a problem of image classification:

<ul>
<li>An image is composed by millions of colored pixels = millions of features.</li>
<li>Each feature brings very little content to the classifier.</li>
</ul>

This is why standard ML classifiers can only work with sophisticated high-level transformations of the image!

<center><img src="./Images/overview_lipacts.png"  /></center>

### An inspiration from the brain: layers!

<center><img src="./Images/vision_system.png"  /></center>

### Convolutional neural networks

<center><img src="./Images/mylenet.png"  /></center>

http://cs231n.github.io/convolutional-networks/

<center><img src="./Images/same_padding_no_strides.gif"  /></center>

Convolutional neural networks (and how to train them) are known since many years, but they have become viable only very recently!

Main reasons:

<ol>
<li>Higher availability of data.</li>
<li>Computational power given by easy-to-program GPUs.</li>
<li>Compositional software frameworks.</li>
</ol>

<center><img src="./Images/main-qimg-3841c6dd04ae33398d5e6743f1072f69.png"  /></center>

<center><img src="./Images/CYHtC4WVAAEyGAL.png"  /></center>

<center><img src="./Images/Slide6.png"  /></center>

## Load data

In [1]:
import tensorflow as tf

In [2]:
from sklearn import datasets, preprocessing, model_selection
data = datasets.load_digits()

In [3]:
X = data['data']
y = data['target']

In [5]:
# PLot a sample
import matplotlib.pyplot as plt
plt.imshow(X[0].reshape(8,8), cmap='gray')
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
import numpy as np
X = preprocessing.MinMaxScaler().fit_transform(X).astype(np.float32)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y)

## Training input function (just like before)

In [8]:
# Define the training input function
def train_input_fcn(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices(({"x": features}, labels))
    dataset = dataset.shuffle(1000).repeat().batch(32)
    return dataset.make_one_shot_iterator().get_next()

## Define the model

In [9]:
def cnn_model_fcn(features, labels, mode):
    
    # Model definition
    
    input_layer = tf.reshape(features["x"], [-1, 8, 8, 1])
    
    # Convolutive & pooling layers
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[3,3], padding="same", activation=tf.nn.relu)
    conv2 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=[3,3], padding="same", activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Final classification layer
    pool1_flat = tf.reshape(pool1, [-1, 4 * 4 * 32])
    dense = tf.layers.dense(inputs=pool1_flat, units=10)
    logits = tf.layers.dense(inputs=dense, units=10)
    
    # Compute the predictions (for PREDICT and EVAL mode)
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        # EstimatorSpec is the object that the model function uses to communicate with the Estimator class
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        # Set up the optimization
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Train the model

In [10]:
# Create the Estimator
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fcn, model_dir="tmp_digits_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tmp_digits_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2ad437b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
classifier.train(input_fn=lambda:train_input_fcn(X_train, y_train), steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into tmp_digits_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.2910864, step = 1
INFO:tensorflow:global_step/sec: 98.3825
INFO:tensorflow:loss = 0.48843643, step = 101 (1.017 sec)
INFO:tensorflow:global_step/sec: 105.029
INFO:tensorflow:loss = 0.29392892, step = 201 (0.952 sec)
INFO:tensorflow:global_step/sec: 106.11
INFO:tensorflow:loss = 0.024376974, step = 301 (0.943 sec)
INFO:tensorflow:global_step/sec: 96.6693
INFO:tensorflow:loss = 0.0839711, step = 401 (1.035 sec)
INFO:tensorflow:global_step/sec: 94.0077
INFO:tensorflow:loss = 0.03691262, step = 501 (1.064 sec)
INFO:tensorflow:global_step/sec: 110.77
INFO:tensorflow:loss = 0.011504129, step = 601 (0.902 sec)
INFO:tensorflow:global_step/sec: 109.404
INF

## Evaluate the model

In [12]:
# Define input function
def eval_input_fcn(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices(({"x": features}, labels))
    dataset = dataset.batch(32) # Note the absence of "repeat"
    return dataset.make_one_shot_iterator().get_next()

In [13]:
print(classifier.evaluate(input_fn=lambda:eval_input_fcn(X_test, y_test)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-01-16:12:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp_digits_convnet_model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-01-16:12:27
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.99333334, global_step = 1000, loss = 0.03369285
{'accuracy': 0.99333334, 'loss': 0.03369285, 'global_step': 1000}
